<a href="https://colab.research.google.com/github/bingsen-wang/PowerElectronics/blob/main/ThermalImpedance_RepeatedPulse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center><h1><b>Thermal Impedance of Repetitive Pulses
</h1></b>

Dr. Bingsen Wang

12/30/2023
</center>

#Transient Thermal Impedance of Single Pulse

In most datasheet of power switching devices, there is transient thermal impedance that depends on the pulse duration of the power losses. The concept of transient thermal impedance may be confusing since it seems that the thermal impedance varies as the power loss input changes. This document explains how the time-independent impance corresonds to the so called transient impedance.

The thermal impedance can be characterized with the Foster RC network.

$$Z_{th}(s) = \sum_{i=1}^{n}\frac{R_i}{R_iC_is + 1}\tag{1}$$

Let the time constant $\tau_i$ for the $i^{th}$ RC network be
$$\tau_i = R_iC_i\tag{2}$$

Then the thermal impedance can be rewritten as
$$Z_{th}(s) = \sum_{i=1}^{n}\frac{R_i}{\tau_is + 1}\tag{3}$$

The unit step response of the junction temperature (rise) is
$$T_{j\_sp}(s) = \frac{1}{s}Z_{th}(s) = \sum_{i=1}^{n}\frac{R_i}{\tau_is^2 + s}=\sum_{i=1}^{n}R_i\left(\frac{1}{s}-\frac{1}{s+{1\over\tau_i}}\right)\tag{4}$$

The transient junction temperature rise in time domain is
$$T_{j\_sp}(t) = \sum_{i=1}^{n}R_i\left(1- e^{-{t\over\tau_i}}\right)\tag{5}$$

The so-called transient thermal impedance essentially takes the junction temperature in response to unit step input as the impedance!

#Thermal Impedance for Repetitive Pulses

For repetitive pulses, let the pulse duration be $T_p$ and the duty radio be $\delta$. Then the frequency of the pulse $f$ is
$$f= \frac{\delta}{T_p}\tag{6}$$
The repetivtive pulse of the power loss can then be modled as
$$p(t) = \Phi\left[\delta - \frac{1}{\pi}\arccos\left(\cos\frac{2\pi\delta}{T_p}t\right)\right]\tag{7}$$

By FFT, the power loss $p(t)$ can be decomposed to
$$p(t) = \sum_{n=-N}^{N}P_n(\omega)e^{jn\omega t}\tag{8}$$
where
$$P_n(\omega) = \frac{\omega}{2\pi}\int_0^{\frac{2\pi}{\omega}}p(t)e^{-jn\omega t}dt\tag{9}$$

Based on superposition, the junction temperature response to the repective-pulse input is
$$T_{j\_rp}(t) = \sum_{n=-N}^{N}P_n(\omega)Z_{th}(jn\omega)e^{jn\omega t}\tag{10}$$

The peak value of $T_{j\_rp}(t)$ is a function of the duty ratio and the pulse duration.
$$T_{j\_peak}(T_p,\delta) = \max_t T_{j\_rp}(t)\tag{11}$$

The Python code plots a family of curves $T_{j\_peak}$ vs $T_p$ corresponding to different values of $\delta$.

#Python Code
The Python code illustrates the real and reactive powers.

In [ ]:
import matplotlib
from os import linesep
from numpy import linspace,logspace,exp,log,fft,pi,arccos,cos,heaviside
import matplotlib.pyplot as plt
from scipy.integrate import quad
from matplotlib import animation,rc
rc('animation', html='jshtml')
plt.rcParams.update({"font.family" : "sans serif","mathtext.fontset" : "cm"})

def Tj_singlePulse(t,Rs,Cs):
  T=0
  for i,R in enumerate(Rs):
    tau = R*Cs[i]
    T += R*(1-exp(-t/tau))
  return T
def pulse(t,Tp,delta):
  return heaviside(delta-1/pi*arccos(cos(2*pi*delta/Tp*t)),0)
def Zth(s,Rs,Cs):
  Z=0
  for i in range(len(Rs)):
    Z += Rs[i]/(Rs[i]*Cs[i]*s+1)
  return Z
def Tj_repetitivePulse(Tp,delta,Rs,Cs):
  Tj_peak=[]
  for k in range(len(Tp)):
    f=delta/Tp[k]
    t=linspace(0,1/f,1024)
    pt = pulse(t,Tp[k],delta)
    pt_fft=fft.rfft(pt)
    Tj_fft=[]
    for i in range(len(pt_fft)):
      Tj_fft.append(pt_fft[i]*Zth(1j*i*2*pi*f,Rs,Cs))
    Tjt = fft.irfft(Tj_fft)
    Tj_peak.append(max(Tjt))
  return Tj_peak

#parameters
clst=['lightskyblue','violet','aqua','cornflowerblue','yellow','w',(.1,.1,.1)] #colors dark bg
# clst=['mediumblue','m','brown','c','r',(.1,.1,.1),'w'] #colors light bg
Nf = 60
t=logspace(-6,0,Nf)
delta_lst = linspace(0.1,0.9,9)
# print(delta_lst)

#RC network
R_lst = [2.748817E-3, 5.715661E-3, 4.153561E-2, 5.616478E-2, 0.3286516, 1.016057, 0.130071]
C_lst = [2.916343E-5, 1.725521E-4, 2.092143E-4, 1.786133E-3, 2.129755E-3, 8.451135E-3, 0.0863404]
Zth_sp = Tj_singlePulse(t,R_lst,C_lst)
Zth_rp = []
for delta in delta_lst:
  Zth_rp.append(Tj_repetitivePulse(t,delta,R_lst,C_lst))

fig = plt.figure(figsize=(16,9),facecolor=clst[-1])
ax_frame = [[0.08, .08, .9, .9], [1e-6,1], [1e-2,10]]# [pos-boundary, xlim, ylim]
ax=fig.add_axes(ax_frame[0],xlim=ax_frame[1], ylim=ax_frame[2],fc='none', xscale='log',yscale='log') #no face color
ax1=fig.add_axes([.7,.1,.3,.5],xlim=[0,1], ylim=[0,1]) #for plotting parameters
ax1.axis('off')
ax.set_xlabel(r'Pulse Duration $T_p\ (s)$',size=20,color=clst[-2])
ax.set_ylabel(r'$Z_{th}\ (^\circ C/W)$',size=20,color=clst[-2])
ax.tick_params(labelcolor=clst[-2],labelsize=16)
ax.grid(True,which='major',color=clst[2], alpha=0.5)
ax.grid(True,which='minor',color=clst[-2], alpha=0.3)
ax.minorticks_on()
for spine in ax.spines.values():
  spine.set_edgecolor(clst[0])
lines_Zth=[]
for i in range(len(delta_lst)+1):
  lines_Zth.append(ax.plot([],[],color=clst[i%5],lw=4)[0])
for i in range(len(R_lst)):
  ax1.text(0,.9-.12*i,'$R_'+str(i+1)+'='+str('{:e}'.format(R_lst[i]))+'$',
                         color=clst[-2],backgroundcolor=(.2,.2,.2),size=16)
  ax1.text(0.5,.9-.12*i,'$C_'+str(i+1)+'='+str('{:e}'.format(C_lst[i]))+'$',
                         color=clst[-2],backgroundcolor=(.2,.2,.2),size=16)
# animation function. This is called sequentially
def animate(i):
  ii = i%Nf
  k = int(i/Nf)
  if k==0:
    if ii==0:
      ax.text(1e-6,Zth_sp[0],'Single Pulse',size=12,backgroundcolor=clst[-2])
    lines_Zth[k].set_data(t[:ii+1],Zth_sp[:ii+1])
  else:
    if ii==0:
      ax.text(1e-6,Zth_rp[k-1][0],r'$\delta='+str(round(delta_lst[k-1],1))+'$',size=14,backgroundcolor=clst[-2])
    lines_Zth[k].set_data(t[:ii+1],Zth_rp[k-1][:ii+1])
  return
anim = animation.FuncAnimation(fig, animate, frames=Nf*10, interval=20)
# anim #uncomment to generate animation in the output area
anim.save("ThermalImpedance_RepetitivePulse.mp4", fps=25, dpi = 120) #uncomment to save